# Timer

In [1]:
from datetime import datetime as dt

start_time = "25.11.2022, 11:15"
start_time = dt.strptime(start_time, "%d.%m.%Y, %H:%M")

while True:
    if start_time < dt.now():
        break
    else:
        sleep(10)

# Parameter optimization

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import sys
sys.path.append('..')

import numpy as np
import pandas as pd
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

ttype = 'buy'
pattern = ['STOCH', 'RSI']#, 'Trend']
indicator_list = pattern
indicator_list_higher = []#['Trend']

work_timeframe = '1h'
higher_timeframe = '4h'
opt_limit = 100000
load = False 

print(f'Timeframe is {work_timeframe}/{higher_timeframe}, trade type is {ttype}')

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Indicator_list'] = indicator_list
configs['Higher_TF_indicator_list'] = indicator_list_higher
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {
                'RSI': {
                        'timeperiod': [14], 
                        'low_bound': [15, 20, 25, 30, 35]
                       },
                'STOCH': {
                          'fastk_period': [9],
                          'slowk_period': [7],
                          'slowd_period': [3], 
                          'low_bound': [10, 15, 20, 25]
                        },
                'Trend': {
                          'timeperiod': [30], 
                          'low_bound': [0]
                        }
             }

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load, 'optimize')

# stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
# if not stat_list:
#     stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
# else:
#     stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')


Timeframe is 1h/4h, trade type is buy
Number of combinations is 20


100%|██████████| 20/20 [26:49<00:00, 80.47s/it] 


# Check local statistics

In [3]:
# rsi + stoch
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg', ascending=False)
stat.head(20)

,pattern,RSI_timeperiod,RSI_low_bound,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
19,STOCH_RSI,14,35,9,7,3,25,6140,0.810475,-3.738333,-1163.683500,-22953.366667,0.8756,0.8772,0.8240,0.8075,0.7987,0.7882,0.7543,0.7588,0.7585,0.7584,0.7664,0.7789,0.7874,0.7939,0.8019,0.8011,0.8044,0.8127,0.8234,0.8334,0.8490,0.8618,0.8663,0.8696,-0.30,-0.59,-0.89,-1.19,-1.49,-1.79,-2.10,-2.41,-2.72,-3.04,-3.37,-3.74,-4.83,-5.09,-5.26,-5.39,-5.50,-5.59,-5.66,-5.71,-5.74,-5.76,-5.77,-5.79
18,STOCH_RSI,14,35,9,7,3,20,4638,0.817667,-4.086250,-845.662000,-18952.027500,0.9053,0.9073,0.8518,0.8211,0.8063,0.7962,0.7579,0.7656,0.7687,0.7658,0.7660,0.7773,0.7866,0.7919,0.8012,0.8000,0.8042,0.8151,0.8250,0.8361,0.8533,0.8686,0.8747,0.8780,-0.31,-0.61,-0.92,-1.24,-1.56,-1.88,-2.21,-2.54,-2.88,-3.22,-3.59,-4.01,-5.38,-5.67,-5.84,-5.97,-6.09,-6.18,-6.25,-6.30,-6.33,-6.35,-6.36,-6.38
17,STOCH_RSI,14,35,9,7,3,15,2501,0.795058,-5.188750,-512.559108,-12977.063750,0.9167,0.9415,0.8686,0.8447,0.8067,0.7957,0.7393,0.7512,0.7519,0.7482,0.7410,0.7532,0.7519,0.7495,0.7536,0.7463,0.7525,0.7692,0.7779,0.7901,0.8137,0.8336,0.8381,0.8463,-0.34,-0.68,-1.02,-1.38,-1.74,-2.12,-2.51,-2.90,-3.30,-3.73,-4.20,-4.78,-7.13,-7.51,-7.70,-7.85,-7.99,-8.09,-8.17,-8.23,-8.26,-8.28,-8.30,-8.32
15,STOCH_RSI,14,30,9,7,3,25,2866,0.800950,-5.291250,-570.477300,-15164.722500,0.9108,0.9049,0.8657,0.8403,0.8256,0.8066,0.7434,0.7507,0.7534,0.7489,0.7454,0.7615,0.7702,0.7711,0.7744,0.7703,0.7819,0.7931,0.8017,0.8058,0.8128,0.8267,0.8277,0.8299,-0.38,-0.76,-1.14,-1.52,-1.90,-2.30,-2.69,-3.10,-3.51,-3.94,-4.40,-4.96,-7.07,-7.46,-7.67,-7.85,-8.00,-8.13,-8.25,-8.32,-8.38,-8.40,-8.42,-8.44
14,STOCH_RSI,14,30,9,7,3,20,2282,0.817592,-5.717083,-416.255817,-13046.384167,0.9025,0.9143,0.8700,0.8453,0.8358,0.8207,0.7513,0.7643,0.7685,0.7641,0.7654,0.7815,0.7907,0.7914,0.7947,0.7918,0.8015,0.8159,0.8227,0.8272,0.8344,0.8526,0.8557,0.8599,-0.39,-0.77,-1.16,-1.56,-1.96,-2.37,-2.79,-3.21,-3.65,-4.11,-4.61,-5.24,-7.81,-8.23,-8.43,-8.60,-8.76,-8.89,-8.99,-9.07,-9.12,-9.15,-9.16,-9.18
13,STOCH_RSI,14,30,9,7,3,15,1389,0.801108,-7.132917,-276.260525,-9907.621250,0.9022,0.9194,0.8496,0.8518,0.8281,0.8171,0.7207,0.7408,0.7375,0.7340,0.7382,0.7591,0.7649,0.7641,0.7633,0.7607,0.7727,0.7959,0.8059,0.8147,0.8245,0.8477,0.8508,0.8629,-0.42,-0.85,-1.27,-1.71,-2.17,-2.63,-3.11,-3.60,-4.12,-4.67,-5.30,-6.14,-10.19,-10.72,-10.92,-11.09,-11.26,-11.39,-11.49,-11.57,-11.61,-11.64,-11.65,-11.67
11,STOCH_RSI,14,25,9,7,3,25,1195,0.835137,-8.361667,-197.010688,-9992.191667,0.8639,0.9024,0.8934,0.8561,0.8403,0.8261,0.8232,0.8481,0.8365,0.8192,0.8136,0.8194,0.8222,0.8222,0.8209,0.7968,0.8027,0.8191,0.8296,0.8349,0.8423,0.8500,0.8314,0.8290,-0.49,-0.99,-1.49,-2.00,-2.52,-3.05,-3.59,-4.15,-4.74,-5.38,-6.10,-7.07,-11.79,-12.42,-12.69,-12.94,-13.19,-13.40,-13.57,-13.70,-13.79,-13.85,-13.87,-13.90
10,STOCH_RSI,14,25,9,7,3,20,970,0.861608,-9.175000,-134.239917,-8899.750000,0.8648,0.9166,0.8966,0.8524,0.8564,0.8446,0.8506,0.8825,0.8651,0.8454,0.8438,0.8501,0.8506,0.8518,0.8559,0.8338,0.8368,0.8553,0.8593,0.8638,0.8697,0.8826,0.8737,0.8764,-0.50,-1.01,-1.53,-2.05,-2.60,-3.15,-3.71,-4.31,-4.94,-5.63,-6.43,-7.53,-13.25,-13.95,-14.20,-14.42,-14.67,-14.86,-15.03,-15.17,-15.26,-15.31,-15.33,-15.36
16

In [2]:
# rsi + stoch + trend
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg', ascending=False)
stat.head(20)

,pattern,RSI_timeperiod,RSI_low_bound,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
19,STOCH_RSI_Trend,14,35,9,7,3,25,30,0,1269,1.175750,-3.446667,223.026750,-4373.820000,1.1477,1.4303,1.3580,1.2511,1.1901,1.2089,1.1771,1.1738,1.1067,1.0857,1.0867,1.1106,1.1104,1.1061,1.1019,1.1010,1.1067,1.1217,1.1502,1.1606,1.2051,1.2347,1.2441,1.2488,-0.33,-0.64,-0.96,-1.28,-1.60,-1.92,-2.25,-2.58,-2.92,-3.26,-3.60,-3.93,-4.25,-4.51,-4.65,-4.75,-4.83,-4.89,-4.93,-4.95,-4.95,-4.93,-4.91,-4.90
18,STOCH_RSI_Trend,14,35,9,7,3,20,30,0,1055,1.144379,-3.681667,152.320021,-3884.158333,1.1776,1.4106,1.3559,1.2459,1.1654,1.1972,1.1740,1.1713,1.1036,1.0797,1.0489,1.0708,1.0680,1.0587,1.0530,1.0488,1.0541,1.0701,1.0936,1.1029,1.1514,1.1825,1.1875,1.1936,-0.33,-0.66,-0.99,-1.33,-1.68,-2.02,-2.37,-2.73,-3.09,-3.46,-3.83,-4.19,-4.54,-4.83,-4.98,-5.09,-5.18,-5.24,-5.30,-5.31,-5.32,-5.31,-5.29,-5.29
17,STOCH_RSI_Trend,14,35,9,7,3,15,30,0,699,1.145487,-4.195417,101.695763,-2932.596250,1.1603,1.4234,1.3567,1.2811,1.1767,1.2147,1.2077,1.1758,1.1199,1.1120,1.0653,1.0878,1.0758,1.0637,1.0497,1.0399,1.0354,1.0481,1.0616,1.0701,1.1391,1.1711,1.1726,1.1832,-0.36,-0.73,-1.09,-1.47,-1.85,-2.24,-2.64,-3.05,-3.47,-3.90,-4.34,-4.78,-5.20,-5.55,-5.72,-5.84,-5.93,-6.01,-6.06,-6.09,-6.10,-6.10,-6.08,-6.09
15,STOCH_RSI_Trend,14,30,9,7,3,25,30,0,450,1.276617,-4.935000,124.477500,-2220.750000,1.2661,1.7468,1.6950,1.5503,1.3642,1.3250,1.3077,1.2794,1.2198,1.2297,1.1871,1.2279,1.2069,1.1936,1.1420,1.1472,1.1605,1.1626,1.1667,1.1683,1.1763,1.2244,1.2385,1.2528,-0.47,-0.91,-1.36,-1.80,-2.25,-2.71,-3.17,-3.64,-4.12,-4.61,-5.11,-5.62,-6.14,-6.57,-6.73,-6.84,-6.93,-7.01,-7.07,-7.10,-7.11,-7.09,-7.05,-7.03
14,STOCH_RSI_Trend,14,30,9,7,3,20,30,0,396,1.333204,-5.157917,131.948850,-2042.535000,1.3111,1.8200,1.7859,1.6602,1.4635,1.4140,1.3945,1.3611,1.2982,1.3015,1.2408,1.2741,1.2512,1.2284,1.1757,1.1848,1.1972,1.2032,1.2050,1.2048,1.2077,1.2615,1.2675,1.2850,-0.47,-0.92,-1.37,-1.84,-2.31,-2.79,-3.28,-3.78,-4.29,-4.81,-5.35,-5.89,-6.46,-6.92,-7.08,-7.19,-7.27,-7.35,-7.41,-7.43,-7.44,-7.42,-7.37,-7.35
16,STOCH_RSI_Trend,14,35,9,7,3,10,30,0,302,1.236112,-5.734583,71.305975,-1731.844167,1.2309,1.5836,1.5752,1.5083,1.3464,1.3503,1.3253,1.2505,1.1588,1.1457,1.0953,1.1732,1.1524,1.1379,1.1044,1.0707,1.0691,1.0980,1.1088,1.1188,1.2491,1.2539,1.2656,1.2945,-0.46,-0.92,-1.39,-1.88,-2.40,-2.94,-3.50,-4.08,-4.67,-5.29,-5.94,-6.58,-7.23,-7.78,-7.98,-8.10,-8.19,-8.27,-8.32,-8.35,-8.36,-8.35,-8.33,-8.32
13,STOCH_RSI_Trend,14,30,9,7,3,15,30,0,296,1.265571,-5.745417,78.608967,-1700.643333,1.3665,1.7588,1.7027,1.6064,1.3841,1.3485,1.3435,1.2866,1.2030,1.1891,1.1382,1.1867,1.1803,1.1704,1.1143,1.1169,1.1134,1.1243,1.1286,1.1297,1.1404,1.2020,1.2103,1.2290,-0.50,-0.99,-1.48,-1.99,-2.51,-3.05,-3.60,-4.17,-4.75,-5.35,-5.97,-6.61,-7.27,-7.81,-7.98,-8.08,-8.16,-8.22,-8.26,-8.27,-8.27,-8.24,-8.19,-8.17
12,STOCH_RSI_Trend,14,30,9,7,3,10,30,0,163,1.268525,-7.353333,43.769575,-1198.593333,1.5149,2.0906,2.0055,1.7954,1.4545,1.3369,1.3049,1.1895,1.0625,1.0651,1.0128,1.1607,1.1606,1.1824,1.1005,1.0800,1.0675,1.1057,1.1125,1.1126,1.1105,1.1193,1.1284,1.1713,-0.59,-1.17,-1.78,-2.40,-3.07,-3.76,-4.48,-5.21,-5.99,-6.79,-7.64,-8.50,-9.45,-10.2

In [3]:
# trend
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg', ascending=False)
stat.head(20)

,pattern,RSI_timeperiod,RSI_low_bound,RSI_vol_window,RSI_vol_q_high,RSI_vol_q_low,RSI_first_candle,RSI_second_candle,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
7,STOCH_RSI_Trend,14,20,48,0.2,0.8,0.8,0.5,9,7,3,25,48,0,186,1.848063,3.287917,157.739625,611.552500,1.3368,1.3714,1.2615,1.2589,1.2331,1.2900,1.3682,1.3992,1.5507,1.6442,1.7687,1.9843,2.1608,2.1477,2.1703,2.2133,2.2364,2.2175,2.2365,2.2436,2.2777,2.2865,2.3194,2.3768,0.32,0.64,0.95,1.26,1.55,1.85,2.15,2.41,2.69,2.96,3.23,3.51,3.76,3.99,4.19,4.37,4.54,4.68,4.82,4.92,4.98,5.03,5.04,5.07
3,STOCH_RSI_Trend,14,15,48,0.2,0.8,0.8,0.5,9,7,3,25,48,0,57,1.636133,3.223750,36.259600,183.753750,1.5764,1.2524,1.1066,1.1051,1.0077,1.0426,1.1779,1.2077,1.2636,1.2917,1.4321,1.6950,1.9504,1.9422,1.9339,2.0608,2.0433,2.0228,2.0429,2.0415,2.0341,1.9978,2.0072,2.0315,0.32,0.63,0.93,1.23,1.51,1.79,2.08,2.36,2.64,2.91,3.19,3.46,3.73,3.95,4.15,4.33,4.49,4.62,4.73,4.82,4.87,4.88,4.87,4.88
6,STOCH_RSI_Trend,14,20,48,0.2,0.8,0.8,0.5,9,7,3,20,48,0,174,1.817696,3.182500,142.279075,553.755000,1.3464,1.3579,1.2337,1.2285,1.1920,1.2499,1.3292,1.3670,1.5292,1.6311,1.7266,1.9486,2.1149,2.1041,2.1224,2.1679,2.1992,2.1830,2.2078,2.2156,2.2519,2.2644,2.2986,2.3548,0.31,0.61,0.91,1.19,1.48,1.76,2.06,2.32,2.59,2.86,3.12,3.40,3.64,3.87,4.06,4.24,4.40,4.54,4.67,4.77,4.84,4.89,4.91,4.94
2,STOCH_RSI_Trend,14,15,48,0.2,0.8,0.8,0.5,9,7,3,20,48,0,56,1.573925,3.181250,32.139800,178.150000,1.5031,1.1387,1.0140,1.0197,0.9329,0.9726,1.1112,1.1420,1.1993,1.2289,1.3716,1.6364,1.8940,1.8878,1.8806,2.0084,1.9918,1.9730,1.9942,1.9931,1.9859,1.9504,1.9600,1.9846,0.31,0.62,0.92,1.21,1.49,1.77,2.05,2.34,2.61,2.88,3.15,3.42,3.68,3.90,4.10,4.28,4.43,4.56,4.67,4.76,4.80,4.81,4.79,4.80
1,STOCH_RSI_Trend,14,15,48,0.2,0.8,0.8,0.5,9,7,3,15,48,0,47,1.666929,3.137083,31.345671,147.442917,1.8162,1.2823,1.0696,1.0712,0.9551,1.0111,1.1798,1.2343,1.3050,1.3351,1.4707,1.7726,2.0679,2.0342,1.9983,2.1243,2.1011,2.0435,2.0437,2.0420,2.0330,1.9951,2.0057,2.0145,0.32,0.63,0.92,1.21,1.49,1.77,2.05,2.33,2.61,2.87,3.14,3.40,3.65,3.86,4.04,4.22,4.36,4.47,4.57,4.65,4.69,4.69,4.67,4.68
11,STOCH_RSI_Trend,14,25,48,0.2,0.8,0.8,0.5,9,7,3,25,48,0,410,1.533267,2.907500,218.639333,1192.075000,1.2323,1.1726,1.1443,1.1736,1.1346,1.1825,1.2395,1.2381,1.3347,1.3667,1.4424,1.5659,1.6633,1.6883,1.7016,1.7277,1.7536,1.7613,1.7972,1.8299,1.8879,1.8798,1.9176,1.9630,0.30,0.58,0.87,1.14,1.40,1.66,1.93,2.17,2.42,2.66,2.90,3.13,3.35,3.55,3.72,3.87,4.01,4.12,4.22,4.29,4.34,4.37,4.38,4.40
5,STOCH_RSI_Trend,14,20,48,0.2,0.8,0.8,0.5,9,7,3,15,48,0,140,1.844983,2.905000,118.297667,406.700000,1.4283,1.3068,1.1896,1.2079,1.1673,1.2354,1.3358,1.3838,1.4449,1.5751,1.6984,1.9873,2.1869,2.1707,2.1901,2.2412,2.2808,2.2598,2.2678,2.2780,2.3201,2.3365,2.3833,2.4038,0.27,0.53,0.79,1.05,1.30,1.56,1.82,2.08,2.33,2.58,2.83,3.09,3.32,3.53,3.72,3.89,4.04,4.17,4.30,4.39,4.47,4.52,4.55,4.59
10,STOCH_RSI_Trend,14,25,48,0.2,0.8,0.8,0.5,9,7,3,20,48,0,370,1.523675,2.796250,193.759750,1034.612500,1.2053,1.1411,1.0992,1.1224,1.0833,1.1331,1.1849,1.1985,1.3070,1.3469,1.4132,1.5462,1.6474,1.6791,1.6961,1.7249,1.7592,1.7736,1.8182,1.8556,1.9310,1.9194,1.9649,2.0177,0.28,0.55,0.81,1.07,1.32,1.57,1.83,2.07,2.31,2.55,2.7

# Research parameter influence

In [16]:
param = 'Trend_timeperiod'

res = stat.groupby([param]).agg({'e_ratio_avg': 'mean',
                                 'pct_price_diff_avg': 'mean',
                                 'e_ratio_rank': 'mean', 
                                 'price_rank': 'mean', 
                                 'forecasts_num': 'sum'}).sort_values(param).reset_index()
res

,Trend_timeperiod,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank,forecasts_num
0,2,3.236211,1.144583,86.068997,44.164444,114
1,3,5.849584,1.552812,62.963132,19.165312,56
2,4,6.061108,1.460000,53.063822,15.588472,32
3,5,6.842036,1.795556,47.689653,14.873611,25
4,6,5.730101,1.770000,37.715247,13.606111,26
5,8,5.441139,1.488750,57.024707,21.980104,61


# Save new config data to config file

In [4]:
from config_updater import ConfigUpdater

ttype = 'buy'
pattern = ['STOCH', 'RSI']
work_timeframe = '15m'
higher_timeframe = '1h'
timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {
                'RSI': {
                        'timeperiod': [14], 
                        'low_bound': [25],
                       },
                'STOCH': {
                          'fastk_period': [9],
                          'slowk_period': [7],
                          'slowd_period': [3], 
                          'low_bound': [25]
                        },
                'Trend': {
                          'timeperiod': [48],
                          'low_bound': [0]
                        }
             }
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)

# Check global statistics

In [2]:
import numpy as np
import pandas as pd
from glob import glob

ttype = 'buy'
pattern = ['STOCH', 'RSI']
work_timeframe = '5m'
higher_timeframe = '1h'
stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
stat = None

for sl in stat_list:
    tmp = pd.read_pickle(sl)
    if stat is None:
        stat = tmp.copy()
    else:
        stat = pd.concat([stat, tmp])
        
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
        
total_stat = stat.groupby(['RSI_timeperiod', 
                           'RSI_low_bound', 
                           'STOCH_fastk_period', 
                           'STOCH_slowk_period', 
                           'STOCH_slowd_period', 
                           'STOCH_low_bound']).agg({'pct_right_forecast_avg': 'mean',
                                                    'pct_price_diff_avg': 'mean',
                                                    'forecast_rank': 'mean', 
                                                    'price_rank': 'mean', 
                                                    'forecasts_num': 'sum'}).sort_values('forecast_rank', 
                                                                                         ascending=False)
total_stat.head(20)

pct_right_forecast_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                           
16             30            5                  4                  3                  10                            75.759735   
                                                                   5                  10                            83.794091   
               35            11                 3                  9                  20                            66.518698   
               25            7                  3                  5                  10                            88.219697   
12             20            7                  3                  5                  10                            86.237538   
               30            11                 3                  9                  25                            67.022344   
               25            5                  4                  5                  10                            79.443447   
14             30            5                  4                  5                  10                            72.603182   
12             20            7                  4                  5                  10                            59.343523   
16             25            9                  3                  5                  10                            73.011894   
                             7                  3                  7                  10                            56.439394   
12             20            7                  3                  7                  10                            65.530303   
16             25            7                  4                  5                  10                            54.286970   
                             5                  3                  7                  10                            58.333333   
12             20            5                  3                  7                  10                            58.333333   
                             9                  3                  5                  10                            73.201098   
               30            5                  4                  5                  10                            69.466932   
16             25            7                  2                  5                  10                            77.102348   
               35            5                  4                  5                  10                            69.892652   
14             20            5                  5                  5                  10                            75.757576   

                                                                                                       pct_price_diff_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                       
16             30            5                  4                  3                  10                         0.843144   
                                                                   5                  10                         1.436705   
               35            11                 3                  9                  20                         0.455156   
               25            7                  3                  5                  10                         1.885038   
12             20            7                  3                  5                  10                         1.803295   
               30            11                 3                  9                  25                         0.418698   
               25            5                  4                  5                  10                         1.366742   
14             30            5                  4                  5                  10                         0.644697   
12            